In [1]:
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
from datetime import datetime
pd.options.display.max_columns = None
#from pybaseball import batting_stats, pitching_stats, cache, playerid_lookup, statcast_batter, statcast_pitcher, statcast
import math
#from fantasy_utils import Fantasy_Projections
import fantasy_utils as fu

import unicodedata
#cache.enable()
#cache.config.cache_type='csv'
#cache.config.save()

In [2]:
from sqlalchemy import MetaData, text, Column, Integer, String, ForeignKey, Table, create_engine, Float, Boolean, DateTime
from sqlalchemy.orm import relationship, backref, sessionmaker
from sqlalchemy.ext.declarative import declarative_base

meta = MetaData()
engine = create_engine('sqlite:///fantasy_data.db', echo=False)
Session = sessionmaker(bind=engine)
session = Session()
#Base = declarative_base()

# Function to correct encoding
def correct_encoding(s):
    return s.encode('iso-8859-1').decode('utf-8')

# Function to correct encoding and remove accents
def remove_accents(input_str):
    input_str = input_str.encode('iso-8859-1').decode('utf-8')
    # Normalizing the string
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    # Filtering out the accent characters and returning the base characters
    return "".join([c for c in nfkd_form if not unicodedata.combining(c)])

In [ ]:
fp = fu.Fantasy_Projections(n_teams=12)
print(fp.yr)
print(fp.proj_systems)
print(fp.pos_hierarchy)

In [ ]:
fp.proj_systems = ['steamer', 'atc', 'thebat'] # used for 2024
#fp.proj_systems = ['dc']

In [ ]:
h, p = fp.prep_projection_data()

In [ ]:
fp.pos_hierarchy = ['C', 'OF', 'SS', '2B', '3B', '1B', 'DH', 'SP', 'RP', 'P']
fp.pos_hierarchy = ['C', 'SS', 'OF', '2B', '3B', '1B', 'DH', 'SP', 'RP', 'P']

In [ ]:
fp.pos_hierarchy = ['RP', 'SP', 'P', 'C', 'DH', '1B', '3B', '2B', 'SS', 'OF']

In [ ]:
df = fp.calc_value()
fp.data['ERA_ly'] = round(fp.data['ERA_ly'],2)
fp.data['Name'] = fp.data['Name'].apply(remove_accents)
df['Name'] = df['Name'].apply(remove_accents)

In [ ]:
df[df['Name']=='Jacob deGrom']

In [ ]:
fp.data.loc[fp.data['playerid']=='21846', ['ERA', 'WHIP', 'SO']] = [3.47, 1.19, 189]

In [ ]:
fp.calc_z(fp.data.loc[fp.data['playerid']=='21846'],'p')

In [ ]:
(260/23) * ((23*12)/df[df['z']>0]['z'].sum())

In [ ]:
4.590441*4.4206898

In [ ]:
df[df['Supp'].notna()][['Name', 'Value', 'Supp', 'z']]

In [ ]:
fp.pos_hierarchy

In [ ]:
#['C', '2B', '3B', 'SS', 'OF', '1B', 'DH', 'SP', 'RP', 'P']
fp.pos_adjust

In [ ]:
#['C', 'OF', 'SS', '2B', '3B', '1B', 'DH', 'SP', 'RP', 'P']
fp.pos_adjust

In [ ]:
fp.pos_adjust

In [ ]:
df[df['z']>0]['z'].sum(), df[df['z']>0].shape[0]

In [ ]:
fp.upload('players2024')

In [ ]:
df[df['Name'].isin(['Jhoan Duran', 'Bryan Abreu', 'Tanner Scott', 'Jason Adam', 'Nico Hoerner'])][['Name', 'Dollars', 'CBS', 'Value']]

In [ ]:
dollars_rem = (12*260) - df['Paid'].sum()
players_rem = (12*23) - df[df['Owner'].notna()].shape[0]
z_rem = df[df['z']>0]['z'].sum() - df[(df['Owner'].notna()) & (df['z']>0)]['z'].sum()
dollars_rem, players_rem, z_rem

In [ ]:
(12*260)/(12*23), dollars_rem/players_rem

In [ ]:
dollars_rem / z_rem

In [ ]:
df[df['Name']=='Cole Ragans']

In [ ]:
df.loc[df['Primary_Pos'].isin(['C', '1B', '2B', '3B', 'OF', 'DH', 'SS']), 'PosClass'] = 'h'
df['PosClass'].fillna('p', inplace=True)

In [ ]:
avg_stats = {}
for a,b in (df[(df['z']>0) & (df['PosClass']=='h')][['HR', 'SB', 'R', 'RBI', 'BA', 'H', 'AB']].sum()/12).reset_index(name='mean').iterrows():
    avg_stats[b['index']] = b['mean']

avg_stats['BA'] = avg_stats['H']/avg_stats['AB']
avg_stats

In [ ]:
df[df['z']>0][['Value', 'CBS', 'Dollars']].sum()

In [ ]:
df['surplus'] = df['Value'] - df['CBS']
df['surplus_rank'] = df['surplus'].rank(pct=True)

In [ ]:
df[df['surplus_rank']>.96].sort_values('surplus_rank', ascending=False)

In [ ]:
df.sort_values('surplus', ascending=False)[['playerid', 'Name', 'Team', 'Pos', 'Value', 'CBS', 'Dollars', 'surplus', 'z', 'HR', 'SB', 'R', 'RBI', 'BA', 'ERA', 'WHIP', 'W', 'SO', 'Sv+Hld']].head(25)

In [ ]:
df[df['Primary_Pos']=='SP'][['Name', 'Pos', 'Primary_Pos', 'Pos_adj', 'Value', 'CBS', 'Dollars', 'surplus']].head(35)

In [ ]:
u = {'21846':{'SO':189, 'ERA':3.47, 'WHIP':1.19, 'W':9, 'SvH':0, 'playerid':'21846'}}
u['21846']

In [ ]:
qry = text("UPDATE players2024 SET ERA=:ERA, WHIP=:WHIP, W=:W, SO=:SO, 'Sv+Hld'=:SvH WHERE playerid=:playerid")

In [ ]:
playerid = '21846'
engine.connect().execute(qry, u[playerid])

In [ ]:
113/96*9
10.59/9*161

In [ ]:
orig_conv =  (tm_dollars/tm_players)*(tot_players/total_z_over_0)

In [ ]:
u[playerid]

In [ ]:
df[df['Keeper']==1][['Name', 'Owner', 'Paid', 'CBS', 'SO', 'Sv+Hld']]

In [ ]:
#['C', '2B', '3B', 'SS', 'OF', '1B', 'DH', 'SP', 'RP', 'P']
fp.pos_adjust

In [ ]:
df['surplus'] = df['Value'] - df['CBS']

In [ ]:
df

In [ ]:
df[df['Primary_Pos']=='2B'][['Name', 'AB', 'H', 'BA', 'HR', 'R', 'RBI', 'SB', 'Pos', 'z', 'Dollars', 'CBS', 'Value', 'Pos_adj']].sort_values('surplus', ascending=False).head(25)

In [ ]:
df[df['Name'].isin(['Nico Hoerner', 'Grayson Rodriguez', 'Justin Steele', 'Corey Seager', 'Josh Naylor', 'Jake Burger', 'Yainer Diaz'])][['Name', 'Pos', 'z', 'surplus', 'Dollars', 'CBS', 'Value', 'Pos_adj', 'AB', 'H', 'BA', 'HR', 'R', 'RBI', 'SB', 'W', 'SO', 'Sv+Hld', 'ERA', 'WHIP']].sort_values('surplus', ascending=False).head(25)

In [ ]:
#engine = create_engine('sqlite:///fantasy_data.db', echo=False)
pd.read_sql('SELECT playerid, Name, Value, Dollars, CBS, Paid, Owner FROM players2024 WHERE Keeper=1', engine)

In [ ]:
df[df['Name'].isin(['Jake Burger', 'Yainer Diaz'])][['BA', 'HR', 'R', 'RBI', 'SB', 'Name', 'Value', 'Dollars', 'CBS']]

In [ ]:
#keepers Google doc
keepers_url = 'https://docs.google.com/spreadsheets/d/1dwDC2uMsfVRYeDECKLI0Mm_QonxkZvTkZTfBgnZo7-Q/edit#gid=1723951361'

In [ ]:
ids = fp.load_id_map()

In [ ]:
# Set Previous Year
prevYear = datetime.now().year-1

In [ ]:
df['Value-CBS'] = df['Value'] - df['CBS']

In [ ]:
c = df[df['Name'].isna()].merge(ids[['IDFANGRAPHS', 'CBSNAME']], left_on='playerid', right_on='IDFANGRAPHS', how='inner')
c['Name'].fillna(c['CBSNAME'],inplace=True)
c.rename(columns={'Name':'new name'},inplace=True)
df = df.merge(c[['playerid','new name']], on='playerid', how='left')
df.Name.fillna(b['new name'],inplace=True)
df.drop(columns='new name', inplace=True)
df.Name.isna().sum()

In [ ]:
df[df['Primary_Pos'].isin(['2B'])].sort_values('Value-CBS', ascending=False).head(18)[['Name', 'z', 'Value-CBS', 'Value', 'CBS', 'Dollars', 'HR', 'SB', 'R', 'RBI', 'BA']].reset_index()

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(name='HR',
        x = ['HR']*df[df['HR']>0].shape[0],
        y = df[df['HR']>0]['HR'].sort_values(ascending=False),
        mode='markers',
    )
)
fig.add_trace(
    go.Scatter(
        name='SB',
        x = ['SB']*df[df['SB']>0].shape[0],
        y = df[df['SB']>0]['SB'].sort_values(ascending=False),
        hovertext=df[df['SB']>0].sort_values('SB',ascending=False)['Name']+'<br>'+df[df['SB']>0].sort_values('SB',ascending=False)['Value'].astype(str),
        mode='markers',
    )
)
fig.show()

In [ ]:
df

In [ ]:
fig = px.violin(df[(df['z']>-1.5) & (df['Primary_Pos'].isin(['C', '1B', '2B', '3B', 'SS', 'OF', 'DH']))], y=["zHR", 'zSB', 'zR', 'zRBI', 'zBA'])
fig.show()

In [ ]:
fig = px.violin(df[(df['z']>-1.5) & (df['Primary_Pos'].isin(['P', 'SP', 'RP']))], y=["zERA", 'zWHIP', 'zW', 'zSO', 'zSv+Hld'])
fig.show()

### StatCast work

In [ ]:
scb = pd.read_csv('data/2022-statcast-h.csv')
scp = pd.read_csv('data/2022-statcast-p.csv')
sc = pd.concat([scb,scp])

In [ ]:
from pybaseball import statcast_batter_exitvelo_barrels, statcast_batter_expected_stats, statcast_batter_percentile_ranks

# get data for all qualified batters in 2023
#d1 = statcast_batter_exitvelo_barrels(2023).set_index('player_id')
#d2 = statcast_batter_expected_stats(2023).set_index('player_id')
#d3 = statcast_batter_percentile_ranks(2023).set_index('player_id')
data = d1.merge(d2.drop(columns=['last_name, first_name', 'year']), left_index=True, right_index=True, how='left')\
    .merge(d3, left_index=True, right_index=True, how='left')

In [ ]:
data.head()

### Working with Drafted Players

In [ ]:
conv = (260/23)*((12*23)/600)
fu.owners(conv)

In [ ]:
df, owners_df = owners(conv)
owners_df

In [ ]:
owners_df['Value'] = owners_df['z']*conv

In [ ]:
owner_list = ['Brewbirds', 'Charmer', 'Dirty Birds', 'Harvey', 'Lil Trump', 'Lima Time', 'Midnight', 'Ugly Spuds', 'Roid Ragers', 'Trouble', 'Wu-Tang', 'Young Guns']
roster = pd.DataFrame(index=['C', '1B', '2B', '3B', 'SS', 'MI', 'CI', 'OF1', 'OF2', 'OF3', 'OF4', 'OF5', 'DH1', 'DH2', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9'], data=np.zeros((23,12)), columns=owner_list)

In [ ]:
owner_list = ['Brewbirds', 'Charmer', 'Dirty Birds', 'Harvey', 'Lil Trump', 'Lima Time', 'Midnight', 'Ugly Spuds', 'Roid Ragers', 'Trouble', 'Wu-Tang', 'Young Guns']
roster = pd.DataFrame(index=['C', '1B', '2B', '3B', 'SS', 'MI', 'CI', 'OF1', 'OF2', 'OF3', 'OF4', 'OF5', 'DH1', 'DH2', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9'], data=np.zeros((23,12)), columns=owner_list)
for tm in owners_df.Owner.to_list():
    for i, row in df[df['Owner']==tm][['Name', 'Owner', 'Primary_Pos', 'Pos', 'Timestamp']].sort_values("Timestamp").iterrows():
        if df.loc[i]['Paid'] > 0:
            check_roster_pos(roster, df.loc[i]['Name'], df.loc[i]['Owner'], df.loc[i]['Primary_Pos'], df.loc[i]['Pos'])

In [ ]:
roster

In [ ]:
def positional_histogram(b):
    fig = go.Figure()
    fig.add_trace(go.Histogram(name='3B', x=b[b['Pos'].str.contains('3B')].sort_values('Value', ascending=False).drop_duplicates('playerid').head(300)['z'], 
                           nbinsx=10, opacity=.5))
    fig.add_trace(go.Histogram(name='1B', x=b[b['Pos'].str.contains('1B')].sort_values('Value', ascending=False).drop_duplicates('playerid').head(300)['z'], 
                           nbinsx=10, opacity=.5))
    fig.add_trace(go.Histogram(name='2B', x=b[b['Pos'].str.contains('2B')].sort_values('Value', ascending=False).drop_duplicates('playerid').head(300)['z'], 
                           nbinsx=10, opacity=.5))
    fig.add_trace(go.Histogram(name='SS', x=b[b['Pos'].str.contains('SS')].sort_values('Value', ascending=False).drop_duplicates('playerid').head(300)['z'], 
                           nbinsx=10, opacity=.5))

    fig.update_layout(title='Z score distribution by position', barmode='overlay', bargap=.1, yaxis_range=[0,55])
    return fig.show()

In [ ]:
positional_histogram(df)

#### Next Closest in Tier

In [ ]:
tier_pos = df[df['Name']=='Freddie Freeman']['Primary_Pos'].iloc[0]
drop_in_tier = round(df[(df['Primary_Pos']==tier_pos) & (df['Owner'].isna())].iloc[1]['Value'] - df[(df['Primary_Pos']==tier_pos) & (df['Owner'].isna())].iloc[0]['Value'],1)
print(drop_in_tier)
df[(df['Primary_Pos']==tier_pos) & (df['Owner'].isna())].iloc[:6][['Name', 'Pos', 'z', 'Value', 'HR', 'SB', 'RBI', 'R', 'BA']]

In [ ]:
i = df[(df['Primary_Pos']=='SS') & (df['playerid']=='18314') & (df['Owner'].isna())].index[0]
v = df[(df['Primary_Pos']=='SS') & (df['Owner'].isna()) & (df['playerid']=='18314')]['Value'].iloc[0]
df[(df['Primary_Pos']=='SS') & (df['Owner'].isna()) & (df['Value']<=v)].iloc[1]['Value']

In [ ]:
df['next_in_tier'] = df.apply(lambda x: next_closest_in_tier(df, x['Primary_Pos'], x['playerid']),axis=1)

In [ ]:
df[(df['Primary_Pos']==tier_pos) & (df['z']>0)]['Value'].hist()

## ------ End of New Work

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from scipy.stats.stats import pearsonr
def scale_data(df, cols):
    """
    INPUT: 
        df: original dataframe
        list: subset of columns to scale
    OUTPUT:
        df: scaled data
    """
    scaler = MinMaxScaler()
    scaler.fit(df[cols])
    scaled_df = scaler.transform(df[cols])
    scaled_df = pd.DataFrame(scaled_df, index=df.index)
    scaled_df.columns=[df[cols].columns.tolist()]
    return scaled_df

In [ ]:
cols = ['BA', 'R', 'RBI', 'HR', 'SB']
scaled_df = scale_data(df[df['Owner'].isna()].set_index('playerid'), cols)
scaled_df.loc['11737', cols]

In [ ]:
def add_distance_metrics(h, player_id, col_list):
    scaled_df = scale_data(h[h['Owner'].isna()].set_index('playerid'), col_list)
    df2 = h[h['Owner'].isna()].loc[:,['playerid', 'Name', 'Pos', 'z']+col_list].set_index('playerid')
    for j, row in scaled_df.iterrows():
        #df2.at[j,'corr'] = pearsonr(scaled_df.loc[player_id,col_list],row[col_list])[0]
        df2.at[j,'eucl_dist'] = np.linalg.norm(scaled_df.loc[player_id,col_list] - row[col_list])
        #df2.at[j,'manh_dist']= sum(abs(e - s) for s, e in zip(scaled_df.loc[player_id,col_list], row[col_list]))
    return df2.sort_values('eucl_dist').iloc[1:6]

In [ ]:
add_distance_metrics(df, '9218', cols)

In [ ]:
mask = (df['Pos'].str.contains('C')) & (df['Owner'].isna())
df[mask][['Name', 'z', 'Dollars', 'HR', 'R', 'RBI', 'SB', 'BA']].sort_values('z', ascending=False).head(10)

In [ ]:
# (Tm 1 $/plyr left / Tm 2 $/plyr left) * suggested price = equivalent comfort level
pp = 5
f'If the price is {pp}, Tm 1 can afford to pay up to {round((10.125 / 6.9) * pp,1)} at the same comfort level as Tm 2'

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df['Dollars'],
    y=df['Value'],
    mode='markers',
    text=df['Name']
)
)
fig.show()

### Fixing 2022 Draft in sqlite db

In [ ]:
url = 'https://xdl.baseball.cbssports.com/draft/results/2022:Pre-season:XDL%20Draft'

In [ ]:
df = pd.read_sql('players', engine)
df.Owner.unique()

In [ ]:
pd.read_sql('players2022', engine).Owner.value_counts()

In [ ]:
tm = 'Lima Time'
df = pd.read_sql('players', engine)
print(df[df['Owner']==tm][['playerid', 'Name', 'Paid']].shape[0])
df[df['Owner']==tm][['playerid', 'Name', 'Paid', 'Value']].sort_values('Paid',ascending=False)

In [ ]:
#player_projections = pd.read_sql("SELECT * FROM players2024", engine)
player_projections = pd.read_excel("C:\\Users\\pddnh\\Documents\\GitHub\\xdl\\data\\PlayerProjections2024.xlsx")
player_projections.rename(columns={'SV':'SV+HLD'},inplace=True)

In [ ]:
hitters = player_projections[~player_projections['Position'].isin(['SP', 'RP', 'P'])]
pitchers = player_projections[player_projections['Position'].isin(['SP', 'RP', 'P'])]

In [ ]:
replacement_stats = {'HR': 10, 'RBI': 50, 'SB': 5, 'AVG': .250, 'R': 50, 'W': 8, 'K': 140, 'ERA': 4.25, 'WHIP': 1.35, 'SV+HLD': 6}

# Assuming player_projections is your DataFrame with player stats

# Step 1: Calculate mean and standard deviation for each stat
stats = ['HR', 'RBI', 'SB', 'AVG', 'R', 'W', 'K', 'ERA', 'WHIP', 'SV+HLD']
for stat in stats:
    player_projections[f'{stat}_mean'] = player_projections[stat].mean()
    player_projections[f'{stat}_std'] = player_projections[stat].std()

# Step 2: Calculate Z-Scores for each stat
for stat in stats:
    if stat in ['ERA', 'WHIP']:  # Inverse stats where lower is better
        player_projections[f'Z_{stat}'] = (player_projections[f'{stat}_mean'] - player_projections[stat]) / player_projections[f'{stat}_std']
    else:
        player_projections[f'Z_{stat}'] = (player_projections[stat] - player_projections[f'{stat}_mean']) / player_projections[f'{stat}_std']

# Step 3: Sum Z-Scores for a Total Value Score
player_projections['Total_Z'] = player_projections[[f'Z_{stat}' for stat in stats]].sum(axis=1)

# Convert Total_Z to auction values, similar to previous methodology
total_z_pool = player_projections[player_projections['Total_Z']>0]['Total_Z'].sum()
player_projections['Auction_Value'] = (player_projections['Total_Z'] / total_z_pool) * (260 * 12)


In [ ]:
total_z_pool

In [ ]:
player_projections.head()

In [ ]:
player_projections[player_projections['Position']=='SP'].head()

In [ ]:
# Update replacement level stats to include pitchers
replacement_stats = {'HR': 10, 'RBI': 50, 'SB': 5, 'AVG': .250, 'R': 50, 'W': 8, 'K': 150, 'ERA': 4.25, 'WHIP': 1.35, 'SV+HLD': 12}

# Separate the dataframe into hitters and pitchers based on your league's position designations
hitters = player_projections[~player_projections['Position'].isin(['SP', 'RP', 'P'])]
pitchers = player_projections[player_projections['Position'].isin(['SP', 'RP', 'P'])]

# Total budget for hitters across all teams
total_budget_for_hitters = 260 * 12 * 0.7
total_budget_for_pitchers = 260 * 12 * 0.3

# Calculate VORP for hitters
for stat in replacement_stats_hitters.keys():
    hitters[f'VORP_{stat}'] = hitters[stat] - replacement_stats_hitters[stat] if stat != 'AVG' else replacement_stats_hitters[stat] - hitters[stat]
hitters['Total_VORP'] = hitters[[f'VORP_{stat}' for stat in replacement_stats_hitters.keys()]].sum(axis=1)

# Calculate VORP for pitchers (handling ERA and WHIP inversely)
for stat in replacement_stats_pitchers.keys():
    pitchers[f'VORP_{stat}'] = pitchers[stat] - replacement_stats_pitchers[stat] if stat not in ['ERA', 'WHIP'] else replacement_stats_pitchers[stat] - pitchers[stat]
pitchers['Total_VORP'] = pitchers[[f'VORP_{stat}' for stat in replacement_stats_pitchers.keys()]].sum(axis=1)

# Combine hitters and pitchers back into one dataframe
player_projections_updated = pd.concat([hitters, pitchers])

# Recalculate the total VORP pool for hitters and pitchers separately
total_vorp_pool_hitters = hitters[hitters['Total_VORP']>=0]['Total_VORP'].sum()
total_vorp_pool_pitchers = pitchers[pitchers['Total_VORP']>=0]['Total_VORP'].sum()

# Calculate the auction dollar value for each player based on their share of the total VORP and the designated budget
hitters['Auction_Value'] = (hitters['Total_VORP'] / total_vorp_pool_hitters) * total_budget_for_hitters
pitchers['Auction_Value'] = (pitchers['Total_VORP'] / total_vorp_pool_pitchers) * total_budget_for_pitchers

# Combine hitters and pitchers back for the final dataframe
player_projections_final = pd.concat([hitters, pitchers])

# Display the updated DataFrame with the Auction_Value column for the top players
player_projections_final[['Player', 'Position', 'Total_VORP', 'Auction_Value']].head()


In [ ]:
# For simplicity, let's calculate VORP based on a simplified model focusing on HR, RBI, SB, AVG, and R for the top players
# Assuming replacement level stats for a generic position player: 10 HR, 50 RBI, 5 SB, .250 AVG, 50 R

# Example replacement level stats
replacement_stats = {'HR': 10, 'RBI': 50, 'SB': 5, 'AVG': .250, 'R': 50}

# Calculate VORP for each category
for stat in replacement_stats.keys():
    player_projections[f'VORP_{stat}'] = player_projections[stat] - replacement_stats[stat]

# Sum the VORP across the selected categories to get a total VORP (this is a simplification for demonstration)
player_projections['Total_VORP'] = player_projections[[f'VORP_{stat}' for stat in replacement_stats.keys()]].sum(axis=1)

# Total budget for hitters across all teams
total_budget_for_hitters = 260 * 12 * 0.7
total_budget_for_pitchers = 260 * 12 * 0.3

# Calculate the total VORP pool for the players in our subset
total_vorp_pool = player_projections['Total_VORP'].sum()
#total_vorp_pool = player_projections[(player_projections['Total_VORP']>=0)]['Total_VORP'].sum()

# Calculate the auction dollar value for each player based on their share of the total VORP
player_projections['Auction_Value'] = (player_projections['Total_VORP'] / total_vorp_pool) * total_budget_for_hitters

# Display the updated DataFrame with the Auction_Value column for the top players
player_projections[['Player', 'Position', 'Total_VORP', 'Auction_Value']].head()
hitters = player_projections[~player_projections['Position'].isin(['SP', 'RP', 'P'])]

In [ ]:
def check_roster_pos(roster, name, team_name, pos, eligible):
    eligible_at = eligible.split('/')
    print(pos)
    eligibility = []
    for p in eligible.split('/'):
        if p=='C':
            eligibility.extend(['C'])
        if p=='1B':
            eligibility.extend(['1B', 'CI'])
        if p=='2B':
            eligibility.extend(['2B', 'MI'])
        if p=='3B':
            eligibility.extend(['3B', 'CI'])
        if p=='SS':
            eligibility.extend(['SS', 'MI'])
        if p=='OF':
            eligibility.extend(['OF1', 'OF2', 'OF3', 'OF4', 'OF5'])
        if p in ['SP', 'RP', 'P']:
            eligibility.extend(['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9'])
        if p in ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10']:
            eligibility.extend([p])
    
    print(eligibility)
    eligibility = list(dict.fromkeys(eligibility))
    if 'SP' in eligible_at or 'RP' in eligible_at: 
        pos_list = eligibility
    else:
        pos_list = eligibility+['DH1', 'DH2']
    for p in pos_list:
        if roster.loc[p, team_name]==0:
            roster.loc[p, team_name] = name
            return p
    
    return pos_list

In [ ]:
n_teams = 12
owner_list = ['Brewbirds', 'Charmer', 'Dirty Birds', 'Harvey', 'Lima Time', "Madness", 'Mother', 'Roid Ragers', 'Trouble', 'Ugly Spuds', 'Wu-Tang', 'Young Guns']
roster = pd.DataFrame(index=['C', '1B', '2B', '3B', 'SS', 'MI', 'CI', 'OF1', 'OF2', 'OF3', 'OF4', 'OF5', 'DH1', 'DH2', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10'], data=np.zeros((33,n_teams)), columns=owner_list)

In [ ]:
roster

In [ ]:
check_roster_pos(roster, 'Justin Steele', 'Lima Time', 'B'+str(int(h[h['Name']=='Justin Steele']['Supp'].iloc[0])), 'B'+str(int(h[h['Name']=='Justin Steele']['Supp'].iloc[0])))

In [ ]:
h = pd.read_sql('players'+str(datetime.now().year), engine)
owners_df = h.query('Paid>0').groupby('Owner').agg({'Name':'count', 'Paid':'sum', 'z':'sum', 'H':'sum', 'AB':'sum', 'HR':'sum', 'R':'sum', 'RBI':'sum', 'SB':'sum', 'Outs':'sum', 'W':'sum', 'SO':'sum', 'Sv+Hld':'sum', 'ER':'sum', 'IP':'sum', 'BB':'sum', 'HA':'sum'}).reset_index()

In [ ]:
h[h['Supp'].notna()][['Name', 'Owner', 'Paid', 'Supp']]

In [23]:
df = pd.read_sql("SELECT * FROM players2024 WHERE Owner IS NOT NULL", engine)

In [24]:
df.groupby('Owner')['playerid'].count()

Owner
Brewbirds      33
Charmer        33
Dirty Birds    33
Harvey         33
Lima Time      33
Madness        33
Mother         33
Roid Ragers    33
Trouble        33
Ugly Spuds     33
Wu-Tang        33
Young Guns     33
Name: playerid, dtype: int64

In [ ]:
df = pd.read_csv("data/2024-dc-proj-p.csv")
df.rename(columns={'PlayerId':'playerid'}, inplace=True)
df.loc[184, 'Pos'] = 'SP'
df.loc[190, 'Pos'] = 'SP'
df.loc[413, 'Pos'] = 'RP'
df.loc[779, 'Pos'] = 'SP'
df.loc[184, 'Primary_Pos'] = 'SP'
df.loc[190, 'Primary_Pos'] = 'SP'
df.loc[413, 'Primary_Pos'] = 'RP'
df.loc[779, 'Primary_Pos'] = 'SP'

In [3]:
df = pd.read_sql("SELECT * FROM players2024", engine)

In [ ]:
df.loc[1183, 'CBSNAME'] = 'Tink Hence'
df.loc[576, 'CBSNAME'] = 'Aldenis Sanchez'
df.loc[655, 'Owner'] = 'Lima Time'
df.loc[772, 'Owner'] = 'Young Guns'
df.loc[576, 'Owner'] = 'Madness'
df.loc[1183, 'Owner'] = 'Dirty Birds'
df.loc[1194, 'Owner'] = 'Wu-Tang'
df.loc[1196, 'Owner'] = 'Ugly Spuds'

In [4]:
df[df['Name'].isin(['Jacob deGrom', 'Jacob Misiorowski', 'Paul Skenes', 'Devin Williams', 'Tink Hence', 'Aldenis Sanchez'])][['Name', 'playerid', 'Owner']]

,Name,playerid,Owner
1196,Devin Williams,15816,None
1197,Jacob deGrom,10954,Lima Time
1198,Paul Skenes,sa3023079,Ugly Spuds
1199,Jacob Misiorowski,sa3020644,Wu-Tang
1200,Tink Hence,sa3014709,Dirty Birds
1201,Aldenis Sanchez,sa3003222,Madness


In [5]:
from sqlalchemy.sql.sqltypes import DATETIME, TIMESTAMP
players = Table('players'+str(datetime.now().year), meta,
                Column('playerid', String, primary_key=True),
                Column('Paid', Integer),
                Column('Supp', Integer),
                Column('Owner', String(25)),
                Column('Timestamp', DATETIME)
    )


In [21]:
conn = engine.connect()
meta.create_all(engine)
price=1
supp=0
owner='Wu-Tang'
playerid='sa3020644'
conn.execute(players.update().values(Paid=price, Supp=supp, Owner=owner, Timestamp=datetime.now()).where(players.c.playerid==playerid))
conn.commit()
conn.close()

In [11]:
with engine.connect() as conn:
    conn.execute(text("UPDATE players2024 SET Owner=NULL WHERE playerid='10954'"))

In [25]:
pd.read_sql("SELECT playerid, Name, Owner, Team, Pos, Supp, Paid, z, Value FROM players2024 WHERE playerid IN ('sa3003222', 'sa3020644', 'sa3023079', 'sa3014709', '15816', '10954')", engine)

,playerid,Name,Owner,Team,Pos,Supp,Paid,z,Value
0,15816,Devin Williams,Young Guns,MIL,RP,0.0,7,0.229674,1.145135
1,10954,Jacob deGrom,Lima Time,TEX,SP,9.0,0,-1.542827,-7.692403
2,sa3023079,Paul Skenes,Ugly Spuds,PIT,SP,1.0,0,-2.110175,-10.521155
3,sa3020644,Jacob Misiorowski,Wu-Tang,MIL,SP,0.0,1,-2.970617,-14.811243
4,sa3014709,Tink Hence,Dirty Birds,STL,SP,10.0,0,-3.363050,-16.767885
5,sa3003222,Aldenis Sanchez,Madness,None,OF,0.0,1,-8.011463,-39.944476


NameError: name 'ids' is not defined